## Step 0: Load Dependencies

In [ ]:
! git clone https://github.com/Eric0801/NTCIR-18-CLIR-pipeline-team6939.git

In [ ]:
!pip3 install transformers rank_bm25 sentence-transformers faiss-cpu jieba tqdm opencc pymupdf 

In [ ]:
!python3 setup_models.py

## Step 1: Intialize Models

if models haven't installed, it will check and install to "models/" directory.

In [ ]:
# Step 0: Initialize Models (Colab Local)
from pathlib import Path
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification

def download_model(name, hf_id, is_classifier=True):
    save_dir = Path("models") / name
    if save_dir.exists() and any(save_dir.iterdir()):
        print(f"[✓] {name} already exists, skipping download.")
        return
    print(f"↓ Downloading {name} from HuggingFace...")
    tokenizer = AutoTokenizer.from_pretrained(hf_id)
    model_cls = AutoModelForSequenceClassification if is_classifier else AutoModel
    model = model_cls.from_pretrained(hf_id)
    save_dir.mkdir(parents=True, exist_ok=True)
    tokenizer.save_pretrained(save_dir)
    model.save_pretrained(save_dir)
    print(f"[✓] {name} saved to {save_dir}")

download_model("zhbert", "hfl/chinese-roberta-wwm-ext", is_classifier=True)
download_model("labse", "sentence-transformers/LaBSE", is_classifier=False)
download_model("cross_encoder", "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2", is_classifier=True)

## Step 2: Extract Paragraphs from PDF (PyMuPDF + EasyOCR fallback)

In [ ]:
import fitz
import easyocr
import json
import os
import numpy as np
from pdf2image import convert_from_path
from pathlib import Path
import traceback
import glob

# Install required poppler-utils if not already installed
!apt-get update -qq && apt-get install -qq -y poppler-utils

# Base directory settings
BASE_DIR = Path("/content/NTCIR-18-CLIR-pipeline-team6939")
PDF_DIRS = [
    BASE_DIR / "pdfs/finance",
    BASE_DIR / "pdfs/insurance"
]
OUTPUT_DIR = BASE_DIR / "outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Initialize EasyOCR
try:
    print("Initializing EasyOCR...")
    reader = easyocr.Reader(['ch_tra', 'en'], gpu=False)
    print("✓ EasyOCR initialized successfully")
except Exception as e:
    print(f"Failed to initialize EasyOCR: {e}")
    print("Will try to process without OCR fallback")
    reader = None

def extract_blocks_with_heuristics(pdf_path, min_block_length=40):
    print(f"Extracting text blocks using PyMuPDF from: {pdf_path}")
    try:
        doc = fitz.open(pdf_path)
        results = []
        doc_id = os.path.splitext(os.path.basename(pdf_path))[0]

        for page_num, page in enumerate(doc):
            blocks = page.get_text("blocks")
            for i, block in enumerate(sorted(blocks, key=lambda b: b[1])):
                x0, y0, x1, y1, text, *_ = block
                clean_text = text.strip().replace("\n", " ")
                if len(clean_text) >= min_block_length:
                    results.append({
                        "pid": f"{doc_id}_p{page_num}_b{i}",
                        "page": page_num,
                        "bbox": [x0, y0, x1, y1],
                        "text": clean_text
                    })

        print(f"✓ Extracted {len(results)} text blocks")
        return results
    except Exception as e:
        print(f"PyMuPDF extraction failed: {e}")
        traceback.print_exc()
        raise

def fallback_ocr_easyocr(pdf_path):
    print(f"Using EasyOCR for fallback OCR processing: {pdf_path}")
    if reader is None:
        print("Cannot perform OCR: EasyOCR not initialized")
        return []

    try:
        images = convert_from_path(pdf_path, dpi=300)
        results = []
        doc_id = os.path.splitext(os.path.basename(pdf_path))[0]

        for page_num, image in enumerate(images):
            print(f"Processing page {page_num + 1}...")

            # Convert PIL Image to numpy array (format that EasyOCR expects)
            image_np = np.array(image)

            # Process with EasyOCR
            try:
                ocr_result = reader.readtext(image_np)
                full_text = " ".join([res[1] for res in ocr_result if len(res[1].strip()) > 0])

                if full_text.strip():
                    results.append({
                        "pid": f"{doc_id}_ocr_{page_num}",
                        "page": page_num,
                        "bbox": None,
                        "text": full_text.strip()
                    })
            except Exception as e:
                print(f"Error during OCR on page {page_num + 1}: {e}")
                continue

        print(f"✓ OCR processing completed, extracted {len(results)} pages of text")
        return results
    except Exception as e:
        print(f"OCR processing failed: {e}")
        traceback.print_exc()
        return []

def process_pdf_file(pdf_path):
    print(f"Processing PDF file: {pdf_path}")

    # Check if file exists
    if not os.path.exists(pdf_path):
        print(f"Error: PDF file does not exist: {pdf_path}")
        return []

    try:
        # First try with PyMuPDF
        segments = extract_blocks_with_heuristics(pdf_path)

        # Skip OCR if PyMuPDF produced good results
        if segments and not all(len(seg['text']) < 40 for seg in segments):
            return segments

        print(f"PyMuPDF extraction results poor or empty, attempting OCR")
        ocr_segments = fallback_ocr_easyocr(pdf_path)

        # If OCR also failed, return whatever we got from PyMuPDF
        if not ocr_segments:
            print("OCR produced no results, returning PyMuPDF results instead")
            return segments

        return ocr_segments
    except Exception as e:
        print(f"Error during PyMuPDF processing: {e}")
        print("Attempting OCR fallback")

        try:
            return fallback_ocr_easyocr(pdf_path)
        except Exception as e2:
            print(f"OCR fallback also failed: {e2}")
            print("Returning empty results for this PDF")
            return []

# Find all PDF files in specified directories
all_pdf_files = []
for pdf_dir in PDF_DIRS:
    if pdf_dir.exists():
        pdf_files = list(pdf_dir.glob("**/*.pdf"))
        all_pdf_files.extend(pdf_files)
        print(f"Found {len(pdf_files)} PDF files in {pdf_dir}")
    else:
        print(f"Warning: Directory does not exist: {pdf_dir}")

print(f"Total PDFs found: {len(all_pdf_files)}")

# Process a subset of PDFs if there are too many (optional)
MAX_PDFS = 99999999999  # Adjust this number as needed
if len(all_pdf_files) > MAX_PDFS:
    print(f"Processing first {MAX_PDFS} PDFs out of {len(all_pdf_files)}")
    all_pdf_files = all_pdf_files[:MAX_PDFS]

# Process all PDF files
all_results = []
successful_pdfs = 0
for pdf_file in all_pdf_files:
    print(f"\nProcessing {pdf_file}... ({successful_pdfs+1}/{len(all_pdf_files)})")
    try:
        pdf_results = process_pdf_file(pdf_file)
        all_results.extend(pdf_results)
        print(f"Extracted {len(pdf_results)} segments from {pdf_file}")
        successful_pdfs += 1
    except Exception as e:
        print(f"Failed to process {pdf_file}: {e}")
        traceback.print_exc()
        continue

# Save all results to a single file
output_file = OUTPUT_DIR / "structured_passages.jsonl"
with open(output_file, "w", encoding="utf-8") as f:
    for r in all_results:
        json.dump(r, f, ensure_ascii=False)
        f.write("\n")

print(f"✅ Extraction completed, results saved to: {output_file}")
print(f"Total extracted passages: {len(all_results)}")
print(f"Successfully processed {successful_pdfs}/{len(all_pdf_files)} PDFs")

# Display some sample text
if all_results:
    print("\nSample texts:")
    for i, r in enumerate(all_results[:3]):  # Show only first 3
        print(f"[{i+1}] {r['pid']}: {r['text'][:100]}...")
else:
    print("No text was extracted from any PDF.")

## Step 3: Hugging Face Translate

In [ ]:
from transformers import MarianMTModel, MarianTokenizer
import torch
import os
import json
from tqdm import tqdm
from opencc import OpenCC  # 新增：簡轉繁

# 下載 HuggingFace 模型 (英文 ➔ 中文)
model_name = "Helsinki-NLP/opus-mt-en-zh"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# 初始化簡轉繁工具
cc = OpenCC('s2t')  # Simplified Chinese ➔ Traditional Chinese

# 載入 queries
with open("/content/NTCIR-18-CLIR-pipeline-team6939/data/translated_query.json", "r", encoding="utf-8") as f:
    queries = json.load(f)

# 設定 cache 檔案路徑（根據 model 名稱自動命名）
cache_path = f"/content/NTCIR-18-CLIR-pipeline-team6939/outputs/translated_cache_{model_name.replace('/', '_')}.json"
if os.path.exists(cache_path):
    with open(cache_path, "r", encoding="utf-8") as f:
        translated_cache = json.load(f)
else:
    translated_cache = {}

# 定義 NMT 翻譯函式（支援 cache + 自訂是否轉繁體）
def translate_with_nmt(query_en, convert_to_traditional=True):
    if query_en in translated_cache:
        return translated_cache[query_en]
    try:
        inputs = tokenizer(query_en, return_tensors="pt", padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            translated = model.generate(**inputs, max_length=512, num_beams=5)
        result = tokenizer.decode(translated[0], skip_special_tokens=True)
        if convert_to_traditional:
            result = cc.convert(result)  # 自動簡體轉繁體
        translated_cache[query_en] = result
        return result
    except Exception as e:
        print(f"Translation error: {e}")
        return ""

# 執行批次翻譯
translated_output = []
for item in tqdm(queries):
    zh = translate_with_nmt(item["query_en"], convert_to_traditional=True)  # 這邊可以控管要不要轉繁體
    item["query_zh_nmt"] = zh
    translated_output.append(item)

# 存檔：翻譯後查詢 + 快取
os.makedirs("/content/NTCIR-18-CLIR-pipeline-team6939/outputs", exist_ok=True)

with open("/content/NTCIR-18-CLIR-pipeline-team6939/outputs/translated_query_nmt.json", "w", encoding="utf-8") as f:
    json.dump(translated_output, f, indent=2, ensure_ascii=False)

with open(cache_path, "w", encoding="utf-8") as f:
    json.dump(translated_cache, f, indent=2, ensure_ascii=False)

print("✅ HuggingFace NMT 簡繁轉換翻譯完成並成功快取保存。")

## Step 4: Run Retrieval (4 Models with Runtime Logging)

In [ ]:

%cd /content/NTCIR-18-CLIR-pipeline-team6939
from run_all_retrievals import run_all_retrievals
run_all_retrievals()

#### 目前 zhbert reranker 跑不出來，但因為你說要 fine tune 所以我先跳過他

## Step 5: 合併結果成 retrieval_rankings.json ，然後評分（評分的 code 有大改，因為原本切太細了，等你修好之後 evaluate 也要再改）

In [ ]:
import json
from collections import defaultdict
from pathlib import Path
import sys

# ──────────────────────────────────────────────────
# 1️⃣ 設定路徑（把 retrieval_rankings.json 放到 outputs/runs）
# ──────────────────────────────────────────────────
BASE         = Path("/content/NTCIR-18-CLIR-pipeline-team6939")
RUNS_DIR     = BASE/"outputs"/"runs"
OUT_RANK     = RUNS_DIR/"retrieval_rankings.json"
GROUND_TRUTH = BASE/"data"/"ground_truths_example.json"
CSV_OUT      = BASE/"outputs"/"evaluation_summary.csv"

# 確保 runs 資料夾存在
RUNS_DIR.mkdir(parents=True, exist_ok=True)

# ──────────────────────────────────────────────────
# 2️⃣ 讀三條 pipeline 的 jsonl，group by model→qid→[pid]
# ──────────────────────────────────────────────────
retrieval = {}
for model_name in ["bm25_only", "dense_dual_encoder", "cross_encoder"]:
    fn       = RUNS_DIR/f"{model_name}.jsonl"
    grouping = defaultdict(list)
    with open(fn, 'r', encoding='utf-8') as f:
        for line in f:
            r = json.loads(line)
            grouping[str(r["qid"])].append(r["pid"])
    retrieval[model_name] = grouping

# ──────────────────────────────────────────────────
# 3️⃣ 寫出 retrieval_rankings.json 到 outputs/runs
# ──────────────────────────────────────────────────
with open(OUT_RANK, 'w', encoding='utf-8') as f:
    json.dump(retrieval, f, indent=2, ensure_ascii=False)
print(f"✅ grouped rankings saved to {OUT_RANK}")

# ──────────────────────────────────────────────────
# 4️⃣ 呼叫 evaluate_all_models()
# ──────────────────────────────────────────────────
# 把 src 加到 module path
sys.path.append(str(BASE/"src"))

from evaluation.evaluation_summary import evaluate_all_models

df = evaluate_all_models(
    ranking_path=str(OUT_RANK),
    ground_truth_path=str(GROUND_TRUTH),
    output_csv_path=str(CSV_OUT),
    ks=[10, 100]
)
print(df)


                Model  MRR  Recall@10  NDCG@10  Recall@100  NDCG@100
0           bm25_only  0.0        0.0      0.0         0.0       0.0
1  dense_dual_encoder  0.0        0.0      0.0         0.0       0.0
2       cross_encoder  0.0        0.0      0.0         0.0       0.0

## Step 6: Translation Error Impact Analysis (還沒試過)

In [ ]:

from translate_error_analysis import extract_translation_impact

impact = extract_translation_impact(
    queries_path="data/translated_query.json",
    predictions_path="outputs/retrieval_rankings.json",
    ground_truth_path="data/ground_truths_example.json"
)

for category, group in impact.items():
    print(f"\n== {category.upper()} ({len(group)} samples) ==")
    for qid, en, zh, pred, gt in group[:1]:
        print(f"QID: {qid}\nEN: {en}\nZH: {zh}\nPRED: {pred}\nGT: {gt}\n---")
